Assignment 1 :
Movie Review Classification using KNN.


1. Import Libraries.


In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import *
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import re



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2. Open the Train Data file and Remove Punctuations and Special Characters.

In [2]:
# Cleaning the train data.

Train_data = open("train_new.csv")  #opens the train file as csv
train_lines = Train_data.readlines() # readlines from the csv file so we get the csv file with both ratings and reviews.

stop_words = set(stopwords.words('english')) #stop_words is a set function that contains all stop words. Efficient for searching.


Movie_ratings = [] # list that will contain all the movie ratings from the train data file.
Train_Movie_reviews = [] # list that will contain all the movie reviews from the train data file.
Train_Cleaned_Movie_Review = [] # list that will contain the cleaned and processed reviews from train data.



clean_data = lambda x: re.sub(re.compile('<.*?>|[^\w+\']|[0-9]'), ' ', x) # removes all the punctuations, numbers, html tags and replaces them with space
clean_data_special = lambda x: re.sub(re.compile('[^\w\s+]'), '', x) # removes all whitespaces and spaces within the text




for line in train_lines:                     # reads line from the file.
  split_line = line.lower().rsplit('\t')  # makes the entire line lowercase and splits each line based on tab.
  Movie_ratings.append(split_line[0]) # takes the first element of the array and appends it to movie ratings list
  Train_Movie_reviews.append(split_line[1]) # takes the second element of the array and appends it to movie reviews list.
  Train_Unclean_Movie_reviews = split_line[1] # assigning the second element of the array to variable Train_Unclean_Movie_reviews.
  Train_Cleaned_Movie_Review.append(clean_data_special(clean_data(Train_Unclean_Movie_reviews.replace('#EOF','')))) # Pass the Train_Unclean_Movie_reviews to the cleaned data function to remove all the unwanted punctuations and spaces.


dict = {"Movie Ratings":Movie_ratings, "Movie Reviews": Train_Movie_reviews, "Cleaned data": Train_Cleaned_Movie_Review} #created a dictionary of movie ratings, reviews and the cleaned data

Train_df = pd.DataFrame(dict) # converted the dictionary to DataFrame.
Train_df = Train_df.dropna() # drop the missing values in the DataFrame.
Train_df = Train_df.reset_index(drop=True) # reset the indices of elements after removing missing values.




3. Remove the Stop Words from Train data

In [3]:
# function to remove all the stop words from the cleaned data.
Train_df['Remove Stop Words'] = Train_df['Cleaned data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))



4. Using Lemmatization and POS Tagging

---

In this cell below it throws an error sometimes for that simply comment the !pip install pattern line at the top and run the cell again. 


In [6]:
#!pip install pattern
import pattern
from nltk.corpus import wordnet as wn
from pattern.en import tag

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer() # Using WordNetLemmatizer to assign meaning to each word in the review.

def pos_tag_text(text): # function for POS Tagging on which we will use our WordNetLemmatizer.
    
    def penn_to_wn_tags(pos_tag):
        if pos_tag.startswith('J'):
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None
    
    tagged_text = tag(text)
    tagged_lower_text = [(word.lower(), penn_to_wn_tags(pos_tag))
                         for word, pos_tag in
                         tagged_text]
    return tagged_lower_text
    
# lemmatize text based on POS tags    
def lemmatize_text(text):
    pos_tagged_text = pos_tag_text(text)
    lemmatized_tokens = [lemmatizer.lemmatize(word, pos_tag) if pos_tag else word for word, pos_tag in pos_tagged_text]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

Train_df['POS tagged'] = Train_df['Remove Stop Words'].apply(lemmatize_text)
Train_df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Movie Ratings,Movie Reviews,Cleaned data,Remove Stop Words,POS tagged
0,+1,one of my all-time favorite so-laughably-lousy...,one of my all time favorite so laughably lousy...,one time favorite laughably lousy totally lova...,one time favorite laughably lousy totally lova...
1,-1,"i had high hopes for this film, because i thou...",i had high hopes for this film because i thou...,high hopes film thought clean shaven kerrigans...,high hop film think clean shaven kerrigans fir...
2,-1,"when this was released, i thought this was one...",when this was released i thought this was one...,released thought one profane films ever made h...,release thought one profane film ever make how...
3,-1,i just watched this movie on starz. let me go ...,i just watched this movie on starz let me go ...,watched movie starz let go things thought coul...,watched movie starz let go thing thought could...
4,+1,i loved it so much that i bought the dvd and t...,i loved it so much that i bought the dvd and t...,loved much bought dvd novel time chemistry act...,love much buy dvd novel time chemistry actor i...


In [ ]:
Train_df['Split'] = [word_tokenize(line1) for line1 in Train_df['POS tagged']]
Train_df

,Movie Ratings,Movie Reviews,Cleaned data,Remove Stop Words,POS tagged,Split
0,+1,one of my all-time favorite so-laughably-lousy...,one of my all time favorite so laughably lousy...,one time favorite laughably lousy totally lova...,one time favorite laughably lousy totally lova...,"[one, time, favorite, laughably, lousy, totall..."
1,-1,"i had high hopes for this film, because i thou...",i had high hopes for this film because i thou...,high hopes film thought clean shaven kerrigans...,high hop film think clean shaven kerrigans fir...,"[high, hop, film, think, clean, shaven, kerrig..."
2,-1,"when this was released, i thought this was one...",when this was released i thought this was one...,released thought one profane films ever made h...,release thought one profane film ever make how...,"[release, thought, one, profane, film, ever, m..."
3,-1,i just watched this movie on starz. let me go ...,i just watched this movie on starz let me go ...,watched movie starz let go things thought coul...,watched movie starz let go thing thought could...,"[watched, movie, starz, let, go, thing, though..."
4,+1,i loved it so much that i bought the dvd and t...,i loved it so much that i bought the dvd and t...,loved much bought dvd novel time chemistry act...,love much buy dvd novel time chemistry actor i...,"[love, much, buy, dvd, novel, time, chemistry,..."
...,...,...,...,...,...,...
24995,+1,it was a doubly interesting experience. for so...,it was a doubly interesting experience for so...,doubly interesting experience reason greatest ...,doubly interesting experience reason great sci...,"[doubly, interesting, experience, reason, grea..."
24996,-1,wow what a great premise for a film : set it a...,wow what a great premise for a film set it a...,wow great premise film set around film maker w...,wow great premise film set around film maker w...,"[wow, great, premise, film, set, around, film,..."
24997,-1,a lot of death happens in the wild. you don't ...,a lot of death happens in the wild you dont n...,lot death happens wild dont need rocket scient...,lot death happen wild dont need rocket scienti...,"[lot, death, happen, wild, dont, need, rocket,..."
24998,+1,corean cinema can be quite surprising for an o...,corean cinema can be quite surprising for an o...,corean cinema quite surprising occidental audi...,corean cinema quite surprising occidental audi...,"[corean, cinema, quite, surprising, occidental..."


5. Read the Test Data file and remove punctuations and special characters.

In [7]:
Test_data = open("1629732045_8755195_test_new.txt", "r")
test_lines = Test_data.readlines()

Test_Movie_Reviews=[]
Test_Cleaned_Movie_Review=[]

clean_data = lambda x: re.sub(re.compile('<.*?>|[^\w+\']|[0-9]'), ' ', x)
clean_data_special = lambda x: re.sub(re.compile('[^\w\s+]'), '', x)

for line in test_lines:
  split_line = line.lower().rsplit('\t')
  Test_Movie_Reviews.append(split_line[0])
  Test_Unclean_Movie_reviews = split_line[0] 
  Test_Cleaned_Movie_Review.append(clean_data_special(clean_data(Test_Unclean_Movie_reviews.replace('#EOF',''))))


dict = {"Movie Reviews": Test_Movie_Reviews, "Cleaned data": Test_Cleaned_Movie_Review}
Test_df = pd.DataFrame(dict)
Test_df = Test_df.dropna()
Test_df = Test_df.reset_index(drop=True)
Test_df.head()


,Movie Reviews,Cleaned data
0,"this is a very low budget film, set in one loc...",this is a very low budget film set in one loc...
1,one minute into the untold and it`s already ri...,one minute into the untold and it s already ri...
2,i recently purchased this on dvd as i hadn't h...,i recently purchased this on dvd as i hadnt he...
3,some people have the ability to use only 3 neu...,some people have the ability to use only neu...
4,"as i've said in the title of this review, it p...",as ive said in the title of this review it pa...


6. Remove the stop words from the Test Data

In [8]:
Test_df['Remove Stop Words'] = Test_df['Cleaned data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


In [9]:
Test_df['POS tagged'] = Test_df['Remove Stop Words'].apply(lemmatize_text)
Test_df.head()

,Movie Reviews,Cleaned data,Remove Stop Words,POS tagged
0,"this is a very low budget film, set in one loc...",this is a very low budget film set in one loc...,low budget film set one location valley shield...,low budget film set one location valley shield...
1,one minute into the untold and it`s already ri...,one minute into the untold and it s already ri...,one minute untold already ripped techniques bl...,one minute untold already rip technique blair ...
2,i recently purchased this on dvd as i hadn't h...,i recently purchased this on dvd as i hadnt he...,recently purchased dvd hadnt heard like robert...,recently purchase dvd hadnt hear like robert c...
3,some people have the ability to use only 3 neu...,some people have the ability to use only neu...,people ability use neurons one eating one brea...,people ability use neurons one eat one breathe...
4,"as i've said in the title of this review, it p...",as ive said in the title of this review it pa...,ive said title review pains say hitch reaches ...,ive say title review pain say hitch reach zeni...


In [ ]:
Test_df['Split'] = [word_tokenize(line1) for line1 in Test_df['POS tagged']]
Test_df

,Movie Reviews,Cleaned data,Remove Stop Words,POS tagged,Split
0,"this is a very low budget film, set in one loc...",this is a very low budget film set in one loc...,low budget film set one location valley shield...,low budget film set one location valley shield...,"[low, budget, film, set, one, location, valley..."
1,one minute into the untold and it`s already ri...,one minute into the untold and it s already ri...,one minute untold already ripped techniques bl...,one minute untold already rip technique blair ...,"[one, minute, untold, already, rip, technique,..."
2,i recently purchased this on dvd as i hadn't h...,i recently purchased this on dvd as i hadnt he...,recently purchased dvd hadnt heard like robert...,recently purchase dvd hadnt hear like robert c...,"[recently, purchase, dvd, hadnt, hear, like, r..."
3,some people have the ability to use only 3 neu...,some people have the ability to use only neu...,people ability use neurons one eating one brea...,people ability use neurons one eat one breathe...,"[people, ability, use, neurons, one, eat, one,..."
4,"as i've said in the title of this review, it p...",as ive said in the title of this review it pa...,ive said title review pains say hitch reaches ...,ive say title review pain say hitch reach zeni...,"[ive, say, title, review, pain, say, hitch, re..."
...,...,...,...,...,...
24995,this movie looked like the out-takes of the de...,this movie looked like the out takes of the de...,movie looked like takes deleted scenes high sc...,movie look like take delete scene high school ...,"[movie, look, like, take, delete, scene, high,..."
24996,this is one of my three all-time favorite movi...,this is one of my three all time favorite movi...,one three time favorite movies quibble directo...,one three time favorite movie quibble director...,"[one, three, time, favorite, movie, quibble, d..."
24997,most families will recognise similarities betw...,most families will recognise similarities betw...,families recognise similarities family created...,family recognise similarity family create bril...,"[family, recognise, similarity, family, create..."
24998,i stumbled upon this movie by accident. i mean...,i stumbled upon this movie by accident i mean...,stumbled upon movie accident mean else could f...,stumble upon movie accident mean else could fi...,"[stumble, upon, movie, accident, mean, else, c..."


7. Vectorization

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(norm = 'l2')
    
train_matrix= vectorizer.fit_transform(Train_df['POS tagged'])
test_matrix = vectorizer.transform(Test_df['POS tagged'])

8. K-Fold Cross Validation

I used the K-Fold cross validation to check for what KNN-K value, I get approximate 50% accuracy. This Validation I implemented before my own implementation of KNN so for this I used the sklearn KNeighborsClassifier. Here I tried for different values of k and approximately for k = 341, I got 0.8056 accuracy and with the same k value for my Knn implementation I got a score of 0.79 on miner.


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    train_matrix, Train_df['Movie Ratings'], test_size=0.3, random_state=123)

In [13]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=341, metric='cosine')
clf = neigh.fit(X_train, y_train)
#clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("KNN Accuracy:",metrics.accuracy_score(y_test, predicted))


KNN Accuracy: 0.8056


In [14]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
cv = KFold(n_splits=25, shuffle = True)
X, y = make_classification(n_samples=450, n_features=7450, n_informative=15, n_redundant=5, random_state=1)
scores = cross_val_score(clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
max(scores)


0.8888888888888888

8. KNN Implementation

In [18]:
from sklearn.metrics.pairwise import cosine_similarity 
import math


similarities = cosine_similarity(test_matrix, train_matrix) # computes cosine similarity between the two as a numpy array

 

def predict_neighbor(nearestNeighbors, labels): # Finds majority vote among the nearest neiighbors.
    
    total = 0
    for neighbor in nearestNeighbors:
      total += int(labels[neighbor])
    if total > 0:
      return 1
    else:
      return -1

prediction_result = []

def KNN(k): # implementation of KNN logic
    
    for similar in similarities:
      knn = np.argsort(-similar)[:k]
      prediction = predict_neighbor(knn, Train_df['Movie Ratings'])
      if prediction == 1:
        prediction_result.append('+1')
      else:
        prediction_result.append('-1')

KNN(341)


output_file = open('KNN_Prediction.txt', 'w')
output_file.writelines( "%s\n" % item for item in prediction_result )
output_file.close()
